In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_list = []

csv_files = [f for f in os.listdir('data/')] 
for file in csv_files:
    file_path = os.path.join('data/', file)
    data_list.append(pd.read_csv(file_path))

df = pd.concat(data_list, ignore_index=True)

df[df['class'] == 'back-slouch'] #Grabs only a specific class.

    

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z32,v32,x33,y33,z33,v33,neck_tilt,shoulder_tilt,mouth_tilt,eye_tilt
0,back-slouch,0.509448,0.524600,-0.522874,0.999766,0.522115,0.491173,-0.499200,0.999721,0.528078,...,0.017744,0.025120,0.437206,1.868234,0.106335,0.011936,30.468780,112.674779,139.763642,142.459910
1,back-slouch,0.514575,0.521808,-0.545932,0.999773,0.526927,0.488849,-0.514747,0.999734,0.534232,...,0.158668,0.028173,0.440444,1.970423,0.146236,0.015577,30.317279,115.427690,128.360529,130.256377
2,back-slouch,0.515132,0.520802,-0.558694,0.999788,0.528085,0.488747,-0.520031,0.999754,0.535698,...,0.152009,0.026906,0.443615,2.081633,0.108856,0.015270,30.678226,113.655348,128.445598,130.600115
3,back-slouch,0.517845,0.519300,-0.536412,0.999783,0.531239,0.487632,-0.492078,0.999755,0.538949,...,0.182745,0.026613,0.453008,2.142330,0.090229,0.016016,30.861230,112.990364,131.319567,130.669615
4,back-slouch,0.519888,0.519069,-0.510512,0.999778,0.533118,0.487201,-0.465184,0.999753,0.540787,...,0.154047,0.024961,0.456965,2.190049,0.033289,0.015050,31.096813,111.697687,130.607744,130.688743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1838,back-slouch,0.498907,0.485181,-0.890447,0.999928,0.516269,0.425195,-0.855742,0.999890,0.529880,...,0.037512,0.004174,0.494592,2.737751,-0.057812,0.003163,28.987877,96.443020,88.707382,91.607098
1839,back-slouch,0.502523,0.485288,-0.917035,0.999930,0.521529,0.426278,-0.884933,0.999891,0.535444,...,0.058680,0.004305,0.508806,2.738026,-0.051352,0.003267,29.419043,97.890269,91.792675,93.812142
1840,back-slouch,0.504600,0.484431,-0.851256,0.999931,0.523545,0.425751,-0.818438,0.999893,0.537509,...,0.060208,0.004389,0.496827,2.737542,-0.044319,0.003322,29.766594,99.364445,92.955582,96.033654
1841,back-slouch,0.507895,0.484440,-0.847029,0.999927,0.527291,0.426104,-0.812437,0.999887,0.541657,...,0.039465,0.004881,0.502580,2.720030,-0.038508,0.003676,30.182733,101.077949,93.129458,96.266487


In [4]:
X = df.drop('class', axis=1) #Features (no class col)
y = df['class'] #Target value (class col)


In [5]:
#Create train/ test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Train Classification Model

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [7]:
"""pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=200)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(max_iter=200)),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

#Train every model (4) on the data.
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model"""

pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())
model = pipeline.fit(X_train, y_train)


In [8]:
model.predict(X_test)

array(['back-slouch', 'upright', 'upright', ..., 'forward-slouch',
       'forward-slouch', 'forward-slouch'], dtype=object)

# Evaluate & Serialise Models

In [9]:
from sklearn.metrics import accuracy_score
import pickle

In [10]:
"""for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))"""

yhat = model.predict(X_test)
print(f"rf: {accuracy_score(y_test, yhat)}")

rf: 0.9952428642964447


In [11]:
"""# SAVE MODEL #
with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)"""

with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(model, f)

# Make Detections w/ Model

In [12]:
# RELOAD INTO NOTEBOOK #
with open('pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

